In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

In [2]:
def get_model_data(data, sample_size=100, feature_num=40, target_num=5):
    data.dropna()
    data = data.values
    shape = data.shape
    X = np.zeros((shape[0]-sample_size, sample_size, feature_num))
    Y = np.zeros(shape=(shape[0]-sample_size, target_num))
    for i in range(shape[0]-sample_size):
        X[i] = data[i:i+sample_size,0:feature_num]
        Y[i] = data[i+sample_size-1,-target_num:]
    X = X.reshape(X.shape[0], sample_size, feature_num, 1)
    Y = Y-1
    return X,Y

In [4]:
data = pd.read_csv('FI2010_test.csv')
X, Y = get_model_data(data)
Y = Y.astype(int)
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y,test_Y = train_test_split(X, Y, test_size=0.3)

In [5]:
train_X.shape

(22285, 100, 40, 1)

In [6]:
input_tensor = Input(shape=(100,40,1))
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (1,10))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  
tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)
layer_x = layers.Reshape((100,96))(layer_x)
layer_x = LSTM(64)(layer_x)
output = layers.Dense(3, activation='softmax')(layer_x)
model = Model(input_tensor, output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 40, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 20, 16)  48          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 100, 20, 16)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 20, 16)  1040        leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [8]:
opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1)
y = to_categorical(train_Y[:,0])
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_X, y, epochs=50, batch_size=32)

Epoch 1/50
697/697 [==============================] - 100s 143ms/step - loss: 0.5035 - accuracy: 0.6664
Epoch 2/50
697/697 [==============================] - 105s 150ms/step - loss: 0.5033 - accuracy: 0.6663
Epoch 3/50
697/697 [==============================] - 107s 153ms/step - loss: 0.5031 - accuracy: 0.6665
Epoch 4/50
697/697 [==============================] - 96s 137ms/step - loss: 0.5031 - accuracy: 0.6669
Epoch 5/50
697/697 [==============================] - 99s 142ms/step - loss: 0.5030 - accuracy: 0.6672
Epoch 6/50
697/697 [==============================] - 98s 141ms/step - loss: 0.5029 - accuracy: 0.6665
Epoch 7/50
697/697 [==============================] - 99s 142ms/step - loss: 0.5027 - accuracy: 0.6673
Epoch 8/50
697/697 [==============================] - 99s 142ms/step - loss: 0.5029 - accuracy: 0.6667
Epoch 9/50
697/697 [==============================] - 97s 140ms/step - loss: 0.5028 - accuracy: 0.6658
Epoch 10/50
697/697 [==============================] - 99s 141ms/step 